# FinPilot: Hybrid Regime-Switching Trading Model

## FinPilot - Crash Survivability Strategy

This notebook implements a **Hybrid Regime-Switching Model** that combines rule-based risk logic with statistical indicators for crash survivability.

## 1. Setup and Imports

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Custom modules
from data_handler import DataHandler
from features import FeatureEngineer
from regime_detector import RegimeDetector
from strategy import TradingStrategy
from backtester import Backtester
from metrics import Metrics

# Style
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
print('Modules loaded successfully!')

## 2. Data Loading and Preprocessing

Loading **BTC/USD** and **NASDAQ 100** historical data with proper timestamp alignment.

In [ ]:
# Load real BTC and NASDAQ data
handler = DataHandler()
crypto_df, nasdaq_df = handler.load_and_prepare(
    'BTC_USD Bitfinex Historical Data.csv', 
    'Nasdaq 100 Historical Data.csv'
)

print(f"Crypto data: {crypto_df.shape[0]} days")
print(f"  Date range: {crypto_df.index[0].strftime('%Y-%m-%d')} to {crypto_df.index[-1].strftime('%Y-%m-%d')}")
print(f"  Columns: {crypto_df.columns.tolist()}")
print(f"\nNASDAQ data: {nasdaq_df.shape[0]} days")
crypto_df.head()

### Data Quality Checks

In [ ]:
# Check for missing data
print("Missing values:")
print(f"  Crypto: {crypto_df.isnull().sum().sum()}")
print(f"  NASDAQ: {nasdaq_df.isnull().sum().sum()}")

# Price range
print(f"\nBTC Price Range:")
print(f"  Min: ${crypto_df['Close'].min():,.2f}")
print(f"  Max: ${crypto_df['Close'].max():,.2f}")

## 3. Feature Engineering

### Layer 1: The Crash Detector

| Indicator | Description |
|-----------|-------------|
| **DUVOL** | Down-to-Up Volatility ratio - crash precursor |
| **NCSKEW** | Negative Skewness - tail risk indicator |
| **Canary** | NASDAQ drops predict crypto crashes |

In [ ]:
# Generate all features
fe = FeatureEngineer(window=20)
features = fe.generate_all_features(crypto_df, nasdaq_df)

print(f"Features generated: {features.shape[0]} observations, {features.shape[1]} features")
print(f"\nFeature list: {features.columns.tolist()}")
features.describe()

### Feature Correlation Heatmap

In [ ]:
# Correlation matrix
fig, ax = plt.subplots(figsize=(12, 8))
corr_matrix = features.corr()
sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn', center=0, fmt='.2f', ax=ax)
ax.set_title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Regime Detection

### Layer 2: The Logic Engine

| Regime | Trigger | Action |
|--------|---------|--------|
| **Normal** | Default | Trend-following |
| **Crash** | DUVOL > 0.5 OR NASDAQ < -3% | Liquidate 100% |
| **Recovery** | Post-crash | Wait for vol mean reversion |

In [ ]:
# Detect regimes
detector = RegimeDetector(
    duvol_threshold=0.5,
    nasdaq_drop_threshold=-0.03,
    volatility_ratio_threshold=1.0
)
regimes = detector.detect_regimes(features)

print("Regime Distribution:")
print(regimes.value_counts())
print(f"\nCrash periods: {(regimes == 'crash').sum()} days")
print(f"Recovery periods: {(regimes == 'recovery').sum()} days")

In [ ]:
# Visualize regimes with price
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.7, 0.3],
                    subplot_titles=['BTC Price with Regime Overlay', 'Market Regime'])

# Price
fig.add_trace(go.Scatter(x=features.index, y=features['price'], 
                          mode='lines', name='BTC Price', line=dict(color='cyan')), row=1, col=1)

# Regime as numeric
regime_numeric = regimes.map({'normal': 0, 'crash': 1, 'recovery': 0.5})
fig.add_trace(go.Scatter(x=features.index, y=regime_numeric, mode='lines',
                          fill='tozeroy', name='Regime', line=dict(color='orange')), row=2, col=1)

fig.update_layout(height=600, template='plotly_dark', title='Regime-Switching Model')
fig.show()

## 5. Trading Strategy

### Risk Management Features:
- **Stop-loss**: 5% maximum loss per position
- **Volatility-based position sizing**: Reduce exposure during high volatility

In [ ]:
# Run strategy
strategy = TradingStrategy(
    rsi_oversold=30,
    rsi_overbought=70,
    stop_loss_pct=0.05,
    volatility_target=0.02
)
signals = strategy.run_strategy(features, regimes)

print("Position Distribution:")
print(signals['position'].value_counts())
print(f"\nTotal trades: {(signals['signal'].diff() != 0).sum()}")

## 6. Backtesting

Simulating trades with **0.1% slippage** for realistic transaction costs.

In [ ]:
# Run backtest
backtester = Backtester(
    initial_capital=100000,
    slippage_pct=0.001  # 0.1% slippage
)

results = backtester.run_backtest(features, signals)
benchmark = backtester.run_buy_and_hold(features)

# Summary
summary = backtester.generate_summary(results, benchmark)
print("Backtest Summary:")
print("-" * 40)
for k, v in summary.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.2f}")
    else:
        print(f"  {k}: {v}")

In [ ]:
# Equity curve comparison
fig = go.Figure()

fig.add_trace(go.Scatter(x=results.index, y=results['portfolio_value'],
                          mode='lines', name='Strategy', line=dict(color='cyan', width=2)))

fig.add_trace(go.Scatter(x=benchmark.index, y=benchmark['portfolio_value'],
                          mode='lines', name='Buy & Hold', line=dict(color='gray', width=1, dash='dash')))

fig.update_layout(
    title='Strategy vs Buy & Hold Performance',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    template='plotly_dark',
    height=500
)
fig.show()

## 7. Evaluation Metrics

### Mathematical Framework

**Crash Survivability Index (CSI):**

$$CSI = \frac{R_{strategy} - R_f}{\max(Drawdown)}$$

**Sharpe Ratio:**

$$Sharpe = \frac{\mu - r_f}{\sigma}$$

In [ ]:
# Calculate metrics
metrics_calc = Metrics(risk_free_rate=0.02)
equity = results['portfolio_value']
all_metrics = metrics_calc.calculate_all_metrics(equity)

print("=" * 50)
print("EVALUATION METRICS")
print("=" * 50)
for k, v in all_metrics.items():
    print(f"{k:25s}: {v:10.2f}")
print("=" * 50)

In [ ]:
# Drawdown analysis
drawdown, max_dd = backtester.calculate_drawdown(equity)

fig = go.Figure()
fig.add_trace(go.Scatter(x=drawdown.index, y=drawdown * 100,
                          mode='lines', fill='tozeroy', name='Drawdown',
                          line=dict(color='red')))

fig.update_layout(
    title=f'Drawdown Analysis (Max: {max_dd*100:.2f}%)',
    xaxis_title='Date',
    yaxis_title='Drawdown (%)',
    template='plotly_dark',
    height=400
)
fig.show()

## 8. Executive Summary

In [ ]:
print("="*60)
print("EXECUTIVE SUMMARY")
print("="*60)
print(f"""
Model: Hybrid Regime-Switching Strategy
Period: {features.index[0].strftime('%Y-%m-%d')} to {features.index[-1].strftime('%Y-%m-%d')}

PERFORMANCE:
  ├─ Total Return:      {all_metrics['total_return']:.2f}%
  ├─ Annual Return:     {all_metrics['annual_return']:.2f}%
  └─ Benchmark Return:  {summary['benchmark_return']:.2f}%

RISK METRICS:
  ├─ Max Drawdown:      {all_metrics['max_drawdown']:.2f}%
  ├─ Volatility:        {all_metrics['volatility']:.2f}%
  └─ CSI:               {all_metrics['csi']:.2f}

RISK-ADJUSTED:
  ├─ Sharpe Ratio:      {all_metrics['sharpe_ratio']:.2f}
  ├─ Sortino Ratio:     {all_metrics['sortino_ratio']:.2f}
  └─ Calmar Ratio:      {all_metrics['calmar_ratio']:.2f}

TRADING:
  ├─ Total Trades:      {summary['total_trades']}
  └─ Transaction Costs: ${summary['total_transaction_costs']:.2f}
""")
print("="*60)

## 9. Failure Analysis

### Known Limitations:
1. **V-shaped recoveries**: Model may exit too early during quick reversals
2. **Whipsaws**: Rapid regime changes can cause excessive trading
3. **Parameter sensitivity**: DUVOL threshold requires careful tuning

In [ ]:
# Identify potential whipsaw periods
signal_changes = signals['signal'].diff().abs()
rolling_changes = signal_changes.rolling(window=10).sum()
whipsaw_periods = rolling_changes[rolling_changes > 4]

print(f"Potential whipsaw periods (>4 trades in 10 days): {len(whipsaw_periods)}")
if len(whipsaw_periods) > 0:
    print("\nSample whipsaw dates:")
    print(whipsaw_periods.head().index.tolist())